***

*Course:* [Math 535](https://people.math.wisc.edu/~roch/mmids/) - Mathematical Methods in Data Science (MMiDS)  
*Chapter:* 6-Optimization theory and algorithms   
*Author:* [Sebastien Roch](https://people.math.wisc.edu/~roch/), Department of Mathematics, University of Wisconsin-Madison  
*Updated:* Jan 6, 2024   
*Copyright:* &copy; 2024 Sebastien Roch

***

In [ ]:
# IF RUNNING ON GOOGLE COLAB, UNCOMMENT THE FOLLOWING CODE CELL
# When prompted, upload: 
#     * mmids.py
#     * advertising.csv 
# from your local file system
# Files at: https://github.com/MMiDS-textbook/MMiDS-textbook.github.io/tree/main/utils
# Alternative instructions: https://colab.research.google.com/notebooks/io.ipynb

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
# PYTHON 3
import numpy as np
from numpy import linalg as LA
from numpy.random import default_rng
rng = default_rng(535)
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import tensorflow as tf
from tensorflow import keras
import mmids

## Motivating example:  deciphering handwritten digits

We now turn to classification.

Quoting [Wikipedia](https://en.wikipedia.org/wiki/Statistical_classification):

> In machine learning and statistics, classification is the problem of identifying to which of a set of categories (sub-populations) a new observation belongs, on the basis of a training set of data containing observations (or instances) whose category membership is known. Examples are assigning a given email to the "spam" or "non-spam" class, and assigning a diagnosis to a given patient based on observed characteristics of the patient (sex, blood pressure, presence or absence of certain symptoms, etc.). Classification is an example of pattern recognition. In the terminology of machine learning, classification is considered an instance of supervised learning, i.e., learning where a training set of correctly identified observations is available.

We will illustrate this problem on the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. Quoting [Wikipedia](https://en.wikipedia.org/wiki/MNIST_database) again:

> The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. The database is also widely used for training and testing in the field of machine learning. It was created by "re-mixing" the samples from NIST's original datasets. The creators felt that since NIST's training dataset was taken from American Census Bureau employees, while the testing dataset was taken from American high school students, it was not well-suited for machine learning experiments. Furthermore, the black and white images from NIST were normalized to fit into a 28x28 pixel bounding box and anti-aliased, which introduced grayscale levels. The MNIST database contains 60,000 training images and 10,000 testing images. Half of the training set and half of the test set were taken from NIST's training dataset, while the other half of the training set and the other half of the test set were taken from NIST's testing dataset.

Here is a sample of the images:

![MNIST sample images](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

**Figure:** MNIST sample images ([Source](https://commons.wikimedia.org/wiki/File:MnistExamples.png))

We first load the data and convert it to an appropriate matrix representation. The data can be accessed with [`tensorflow.keras.datasets.mnist`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist).

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
mnist = keras.datasets.mnist
(imgs, labels), (test_imgs, test_labels) = mnist.load_data()
len(imgs)

For example, the first image and its label are:

In [ ]:
plt.figure()
plt.imshow(imgs[0])
plt.show()

In [ ]:
labels[0]

For now, we look at a subset of the samples: the 0's and 1's.

To find all such samples, we use a [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [ ]:
i01 = [i for i in range(len(labels)) if (labels[i]==0) or (labels[i]==1)]
imgs01 = imgs[i01]
labels01 = labels[i01]

In this new dataset, the first sample is:

In [ ]:
plt.figure()
plt.imshow(imgs01[0])
plt.show()

In [ ]:
labels01[0]

Next, we transform the images into vectors. For this we use the [`flatten()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) function, which returns a copy of the array collapsed into one dimension.

In [ ]:
X = np.vstack([imgs01[i].flatten() for i in range(len(labels01))])
y = labels01

The input data is now of the form $\{(\mathbf{x}_i, y_i) : i=1,\ldots, n\}$ where $\mathbf{x}_i \in \mathbb{R}^d$ are the features and $y_i \in \{0,1\}$ is the label. Above we use the matrix representation $X \in \mathbb{R}^{d \times n}$ with columns $\mathbf{x}_i$, $i = 1,\ldots, n$ and $\mathbf{y} = (y_1, \ldots, y_n)^T \in \{0,1\}^n$. 

Our goal: 

> to learn a classifier from the examples $\{(\mathbf{x}_i, y_i) : i=1,\ldots, n\}$, that is, a function $\hat{f} : \mathbb{R}^d \to \mathbb{R}$ such that $\hat{f}(\mathbf{x}_i) \approx y_i$.

This problem is referred to as [binary classification](https://en.wikipedia.org/wiki/Binary_classification).

## Background: review of differentiable functions of several variables and introduction to automatic differentiation

**NUMERICAL CORNER:** We illustrate the use of [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) to compute gradients. 

Quoting [Wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation):

> In mathematics and computer algebra, automatic differentiation (AD), also called algorithmic differentiation or computational differentiation, is a set of techniques to numerically evaluate the derivative of a function specified by a computer program. AD exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program. Automatic differentiation is distinct from symbolic differentiation and numerical differentiation (the method of finite differences). Symbolic differentiation can lead to inefficient code and faces the difficulty of converting a computer program into a single expression, while numerical differentiation can introduce round-off errors in the discretization process and cancellation.

We will use the [TensorFlow](https://www.tensorflow.org/overview), specifically [`tensorflow.GradientTape()`](https://www.tensorflow.org/api_docs/python/tf/GradientTape). See [here](https://www.tensorflow.org/guide/autodiff) for a quick introduction. Here is an example.

In [ ]:
import tensorflow as tf

In [ ]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

with tf.GradientTape() as tape:
    f = 3 * x**2 + tf.exp(x) + y

In [ ]:
[df_dx, df_dy] = tape.gradient(f, [x, y])
print(df_dx.numpy())
print(df_dy.numpy())

The input parameters can also be vectors, which allows to consider function of large numbers of variables. 

In [ ]:
z = tf.Variable([1., 2., 3.])

with tf.GradientTape() as tape:
    g = tf.reduce_sum(z**2)

In [ ]:
grad_g = tape.gradient(g, z) # gradient is (2 z_1, 2 z_2, 2 z_3)
print(grad_g.numpy())

Here is another typical example.

In [ ]:
X = tf.Variable(tf.random.normal((3, 2))) # dataset (features)
y = tf.Variable([[1., 0., 1.]]) # dataset (labels)
theta = tf.Variable(tf.ones((2,1))) # parameter assignment

with tf.GradientTape() as tape:
    predict = X @ theta # classifier with parameter vector θ
    loss = tf.reduce_sum((predict - y)**2) # loss function

In [ ]:
grad_loss = tape.gradient(loss, theta)
print(grad_loss.numpy())

$\unlhd$

**NUMERICAL CORNER:** We return to [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation). 

Each component of the output of `gradient(f, x)` is itself a function and can also be differentiated to obtain the second derivative.

In [ ]:
x = tf.Variable(0.0)
y = tf.Variable(0.0)

with tf.GradientTape() as t2:
    with tf.GradientTape() as t1:
        f = x * y + x**2 + tf.exp(x) * tf.cos(y)
    df_dx = t1.gradient(f, x) # needs to be within t2

print(df_dx.numpy()) # answer is 1 (see example is next notebook)

In [ ]:
d2f_dx2 = t2.gradient(df_dx, x) # answer is 3 (see example is next notebook)
print(d2f_dx2.numpy())

$\unlhd$